# Tugas Klasifikasi (Data Mining 2)

Tanggal Pembuatan Awal : 08 November 2024

Tanggal Perbaikan : 14 November 2024

Feedback Tugas 3 :
- bagus pendahuluan dan penjelasan field.
- bagus penjelasan praproses
- judulnya EDA tapi isinya bukan EDA
- apa yang menarik dari describe, perlu ada penjelasan.
- 22 jangan dihardcode
- prec. recall perlu per kelas

### Kelompok 1
1. Alifa Salsabila (2308138)
2. Muhammad Bintang Eighista (2304137)
3. Nina Wulandari (2312091)
4. Putra Aditya Simbolon (2305839)
5. Yahyo Abdullozoda (2313368)

# Latar Belakang
Analisis prediksi perilaku pelanggan dalam sebuah toko online dapat memberikan keuntungan yang signifikan. Dalam konteks ini, mengetahui kemungkinan seorang pengunjung akan melakukan pembelian memungkinkan penyesuaian strategi penjualan. Misalnya, untuk pelanggan dengan peluang tinggi melakukan pembelian, toko dapat menawarkan produk-produk populer untuk mendorong upselling. Sebaliknya, untuk pelanggan dengan kemungkinan rendah, kupon atau insentif lain bisa diberikan guna meningkatkan motivasi belanja. Prediksi ini menggunakan metode data mining, yang memungkinkan analisis data besar dan memberikan jawaban prediktif mengenai peluang suatu pembelian secara efisien. Dalam kompetisi DATA MINING CUP (DMC) 2013, tujuan utamanya adalah untuk meramalkan apakah pengunjung akan melakukan pembelian berdasarkan data sesi dan transaksi historis.

# Tujuan:
1. Membangun model prediksi pesanan menggunakan data sesi historis.
2. Mengenali pola perilaku pelanggan yang meningkatkan kemungkinan pembelian.
3. Menghasilkan prediksi probabilitas (0 hingga 1) untuk setiap sesi, di mana 0 menunjukkan tidak ada pesanan dan 1 menunjukkan pesanan dilakukan.
4. Meminimalkan kesalahan prediksi untuk mendekati hasil aktual.
5. Mendukung optimalisasi strategi toko online guna meningkatkan konversi dan pengalaman belanja pelanggan.

# 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
from copy import deepcopy
warnings.filterwarnings('ignore')

# 2. DATASET YANG DIGUNAKAN

## transact_train
* Deskripsi: File ini adalah data pelatihan yang berisi semua kolom (data arrays) yang dijelaskan dalam dokumen features.pdf, termasuk kolom target order.

* Kolom order: Kolom order menunjukkan apakah sesi tersebut menghasilkan pembelian (y untuk pembelian, n untuk tidak ada pembelian). Ini merupakan target yang ingin diprediksi.

* Kegunaan: Digunakan untuk melatih model. Model akan belajar dari data ini untuk memahami pola-pola yang mengarah pada terjadinya atau tidak terjadinya pembelian.


In [2]:
transact_train = pd.read_csv('DMC//transact_train.csv', sep=',')

In [3]:
#5 Data Pertama
transact_train.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.00,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.94,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39887.00,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15633.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y


In [4]:
# 5 Data Terakhir
transact_train.tail()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
429008,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,n
429009,49999,18,7,675114.0,6,59.0,199.99,509.96,1,89.99,...,?,25038,?,?,?,0,24,1,4,n
429010,49999,18,7,715341.0,7,59.0,649.99,1159.95,1,89.99,...,completely orderable,25038,?,?,?,0,24,1,4,n
429011,50000,18,7,0.0,1,6.99,6.99,6.99,1,6.99,...,?,47,300,537,5,0,53,1,34,n
429012,50000,18,7,82943.0,1,6.99,6.99,6.99,1,6.99,...,completely orderable,47,300,537,5,0,53,1,34,n


In [5]:
# Dimensi Data
transact_train.shape

(429013, 24)

Ukuran dataset terdiri dari 429013 baris dan 24 kolom

In [6]:
# Informasi tentang tipe data dari setiap field pada Dataset
transact_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429013 entries, 0 to 429012
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   sessionNo        429013 non-null  int64  
 1   startHour        429013 non-null  int64  
 2   startWeekday     429013 non-null  int64  
 3   duration         429013 non-null  float64
 4   cCount           429013 non-null  int64  
 5   cMinPrice        429013 non-null  object 
 6   cMaxPrice        429013 non-null  object 
 7   cSumPrice        429013 non-null  object 
 8   bCount           429013 non-null  int64  
 9   bMinPrice        429013 non-null  object 
 10  bMaxPrice        429013 non-null  object 
 11  bSumPrice        429013 non-null  object 
 12  bStep            429013 non-null  object 
 13  onlineStatus     429013 non-null  object 
 14  availability     429013 non-null  object 
 15  customerNo       429013 non-null  object 
 16  maxVal           429013 non-null  obje

Dokumentasi Field Dataset

|No | Nama Kolom | Deskripsi    | Rentang Nilai | Missing Value |
|:-:|:-----------|:-------------|:--------------|:--------------|
|1| sessionNo  | Nomor urut sesi | Bilangan Asli | Tidak  |
|2| startHours | Jam dimulainya sesi | Bilangan asli antara 0 dan 23 | Tidak |
|3| startWeekday| Hari saat sesi dimulai (1=Senin, ..., 7=Minggu) | Bilangan asli dalam {1, 2, 3, 4, 5, 6, 7} | Tidak |
|4| duration| Durasi dalam detik sejak sesi dimulai      | Bilangan desimal tak-negatif        | Tidak |
|5| cCount | Jumlah produk yang diklik | Bilangan asli | Tidak |
|6| cMinPrice | Harga terendah dari produk yang diklik | Bilangan desimal tak-negatif | Ya |
|7| cMaxPrice | Harga tertinggi dari produk yang diklik | Bilangan desimal tak-negatif | Ya |
|8| cSumPrice | Jumlah harga dari semua produk yang diklik | Bilangan desimal tak-negatif | Ya |
|9| bCount | Jumlah produk yang dimasukkan ke keranjang | Bilangan asli | Tidak |
|10| bMinPrice | Harga terendah dari produk di keranjang | Bilangan desimal tak-negatif | Ya |
|11| bMaxPrice | Harga tertinggi dari produk di keranjang | Bilangan desimal tak-negatif | Ya |
|12| bSumPrice | Jumlah harga dari semua produk di keranjang | Bilangan desimal tak-negatif | Ya |
|13| bStep | Langkah pemrosesan pembelian | Bilangan asli dalam {1, 2, 3, 4, 5} | Ya |
|14| onlineStatus | Status online pelanggan (y=ya, n=tidak) | String | Ya |
|15| availability | Status pengiriman | String | Ya |
|16| customerID | ID pelanggan | String | Ya |
|17| maxVal | Harga pembelian maksimum yang diperbolehkan untuk pelanggan | Bilangan asli | Ya |
|18| customerScore | Evaluasi pelanggan dari sudut pandang toko | Bilangan asli | Ya |
|19| accountLifetime | Lama akun pelanggan dalam bulan | Bilangan asli | Ya |
|20| payments | Jumlah pembayaran yang dilakukan oleh pelanggan | Bilangan asli | Ya |
|21| age | Usia pelanggan | Bilangan asli | Ya |
|22| address | Bentuk sapaan pelanggan (1=Tuan, 2=Ny, 3=Perusahaan) | Bilangan asli dalam {1, 2, 3} | Ya |
|23| lastOrder | Waktu dalam hari sejak pesanan terakhir | Bilangan asli | Ya |
|24| order | Hasil sesi (y=pembelian, n=tidak ada pembelian) | Karakter | Tidak |

Kolom kolom yang tipe datanya tidak sesuai diubah manjadi tipe data yang sesuai. 
- kolom cMinPrice dari string ke float
- kolom cMaxPrice dari string ke float
- kolom cSumPrice dari string ke float
- kolom bMinPrice dari string ke float
- kolom bMaxPrice dari string ke float
- kolom bSumPrice dari string ke float
- kolom bStep dari string ke int
- kolom onlineStatus dari string ke category
- kolom availability dari string ke category
- kolom maxVal dari string ke int
- kolom customerScore dari string ke int
- kolom accountLifetime dari string ke int
- kolom payments dari string ke int
- kolom age dari string ke int
- kolom address dari string ke int
- kolom lastOrder dari string ke int
- kolom order dari string ke category

In [7]:
transact_train['sessionNo'] = transact_train['sessionNo'].astype(int)
transact_train['startHour'] = transact_train['startHour'].astype(int)
transact_train['startWeekday'] = transact_train['startWeekday'].astype(int)
transact_train['duration'] = transact_train['duration'].astype(float)
transact_train['cCount'] = transact_train['cCount'].astype(int)
transact_train['cMinPrice'] = pd.to_numeric(transact_train['cMinPrice'], errors='coerce')
transact_train['cMaxPrice'] = pd.to_numeric(transact_train['cMaxPrice'], errors='coerce')
transact_train['cSumPrice'] = pd.to_numeric(transact_train['cSumPrice'], errors='coerce')
transact_train['bCount'] = transact_train['bCount'].astype(int)
transact_train['bMinPrice'] = pd.to_numeric(transact_train['bMinPrice'], errors='coerce')
transact_train['bMaxPrice'] = pd.to_numeric(transact_train['bMaxPrice'], errors='coerce')
transact_train['bSumPrice'] = pd.to_numeric(transact_train['bSumPrice'], errors='coerce')
transact_train['bStep'] = pd.to_numeric(transact_train['bStep'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['onlineStatus'] = transact_train['onlineStatus'].astype('category')
transact_train['availability'] = transact_train['availability'].astype('category')
transact_train['customerNo'] = transact_train['customerNo'].astype('string')
transact_train['maxVal'] = pd.to_numeric(transact_train['maxVal'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['customerScore'] = pd.to_numeric(transact_train['customerScore'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['accountLifetime'] = pd.to_numeric(transact_train['accountLifetime'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['payments'] = pd.to_numeric(transact_train['payments'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['age'] = pd.to_numeric(transact_train['age'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['address'] = pd.to_numeric(transact_train['address'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['lastOrder'] = pd.to_numeric(transact_train['lastOrder'], errors='coerce').fillna(np.nan).astype('Int64')
transact_train['order'] = transact_train['order'].astype('category')


In [8]:
transact_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429013 entries, 0 to 429012
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   sessionNo        429013 non-null  int32   
 1   startHour        429013 non-null  int32   
 2   startWeekday     429013 non-null  int32   
 3   duration         429013 non-null  float64 
 4   cCount           429013 non-null  int32   
 5   cMinPrice        426248 non-null  float64 
 6   cMaxPrice        426248 non-null  float64 
 7   cSumPrice        426248 non-null  float64 
 8   bCount           429013 non-null  int32   
 9   bMinPrice        423883 non-null  float64 
 10  bMaxPrice        423883 non-null  float64 
 11  bSumPrice        423883 non-null  float64 
 12  bStep            237680 non-null  Int64   
 13  onlineStatus     429013 non-null  category
 14  availability     429013 non-null  category
 15  customerNo       429013 non-null  string  
 16  maxVal           275

diatas ialah informasi mengenai kolom yang tipe datanya sudah diubah

In [9]:
data_train = deepcopy(transact_train)

# 3. Pra-Proses

## Cek jumlah dan persentase missing value

In [10]:
# Mengganti setiap nilai pada kolom yang bernilai '?' menjadi None
data_train = data_train.replace('?', None)

In [11]:
print('Jumlah Missing Value')
print(data_train.isnull().sum())

print()

print('Persentase Missing Value')
print(data_train.isnull().sum()/data_train.shape[0] * 100)

Jumlah Missing Value
sessionNo               0
startHour               0
startWeekday            0
duration                0
cCount                  0
cMinPrice            2765
cMaxPrice            2765
cSumPrice            2765
bCount                  0
bMinPrice            5130
bMaxPrice            5130
bSumPrice            5130
bStep              191333
onlineStatus       160379
availability       165255
customerNo         151098
maxVal             153740
customerScore      153740
accountLifetime    153740
payments           151098
age                151396
address            151098
lastOrder          151098
order                   0
dtype: int64

Persentase Missing Value
sessionNo           0.000000
startHour           0.000000
startWeekday        0.000000
duration            0.000000
cCount              0.000000
cMinPrice           0.644503
cMaxPrice           0.644503
cSumPrice           0.644503
bCount              0.000000
bMinPrice           1.195768
bMaxPrice           1.1957

Diatas adalah informasi mengenai missing value dari setiap kolom, bisa dilihat berdasarkan informasi di atas ada beberapa kolom yang perlu di perbaiki missing value nya

## Mengisi Miising Value untuk field bStep

In [12]:
# Fungsi untuk mengisi missing value pada kolom bStep
def fill_bStep(df):
    for i, row in df.iterrows():
        if pd.isna(row['bStep']):  # Cek apakah bStep adalah NaN
            if row['order'] == 'y':
                # Dapatkan indeks dari baris dengan missing bStep untuk 'order' y
                missing_indices = df[(df['order'] == 'y') & (df['bStep'].isna())].index
                if len(missing_indices) == 1:
                    df.at[missing_indices[0], 'bStep'] = 5
                else:
                    # Isi dengan 1 hingga n-1, dan yang terakhir dengan 5
                    for j, idx in enumerate(missing_indices):
                        if j == len(missing_indices) - 1:
                            df.at[idx, 'bStep'] = 5
                        else:
                            df.at[idx, 'bStep'] = j + 1
            elif row['order'] == 'n':
                # Isi semua missing bStep untuk order 'n' dengan 1
                df.loc[(df['order'] == 'n') & (df['bStep'].isna()), 'bStep'] = 1
    return df

# Terapkan fungsi ke DataFrame
data_train = fill_bStep(data_train)

Proses ini bertujuan untuk mengisi nilai missing value yang ada pada kolom bStep di dataset dalam setiap sesi dengan ketentuan berikut:

Untuk order yang bernilai 'y': Jika hanya ada satu nilai missing di kolom bStep, nilai tersebut diisi dengan angka 5. Jika ada lebih dari satu nilai missing, nilai pertama diisi dengan angka 1, nilai berikutnya bertambah satu, dan nilai terakhir diisi dengan angka 5.

Sedangkan, untuk order yang bernilai 'n': Semua nilai missing pada kolom bStep diisi dengan angka 1. Setelah pengisian dilakukan, kita memeriksa kembali apakah masih ada nilai missing yang tersisa di kolom bStep untuk memastikan bahwa proses pengisian telah berhasil.

## Grouping Dataset berdasarkan sesi

In [13]:
def groupData(data, field):
    grouped = data.groupby(field)
    splitted = [group for _, group in grouped]
    return splitted

In [14]:
splitted = groupData(data_train, 'sessionNo')

Selanjutnya kami mengelompokkan data berdasarkan sessionNo. Hasil pengelompokan ini dipecah menjadi beberapa bagian, di mana setiap bagian berisi data yang hanya terkait dengan satu nilai unik dari kolom tersebut. Keluaran akhirnya berupa daftar yang berisi setiap kelompok data tersebut, sehingga mudah diakses atau dianalisis per sessionnya.

## Mengakses Data pada Sesi Tertentu

In [15]:
def get_session_data(splitted, n):
    return splitted[n-1]

In [16]:
# Contoh penggunaan:
get_session_data(splitted, 2) #menampilkan data sesi ke 2

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
3,2,6,5,0.0,0,NaN,NaN,NaN,0,NaN,...,completely orderable,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,y
4,2,6,5,15633.0,0,NaN,NaN,NaN,0,NaN,...,completely orderable,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,y
5,2,6,5,26235.0,0,NaN,NaN,NaN,0,NaN,...,completely orderable,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,y
6,2,6,5,71.2,0,NaN,NaN,NaN,0,NaN,...,completely orderable,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,y
7,2,6,5,94469.0,0,NaN,NaN,NaN,0,NaN,...,completely orderable,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,y


In [17]:
def getCountLog(splitted):
    countLog = []
    for session in splitted:
        countStep = 0
        countLog.append(len(session))
    return countLog

In [18]:
countLog = getCountLog(splitted)

In [19]:
def groupBy(splitted):
    processed_sessions = []
    
    for session in splitted:
        tmpSession = session.iloc[-1].copy()
        
        # Filter nilai non-NA menggunakan notna()
        cMinPrice_filtered = session['cMinPrice'][session['cMinPrice'].notna()].tolist()
        cMaxPrice_filtered = session['cMaxPrice'][session['cMaxPrice'].notna()].tolist()
        cSumPrice_filtered = session['cSumPrice'][session['cSumPrice'].notna()].tolist()
        bMinPrice_filtered = session['bMinPrice'][session['bMinPrice'].notna()].tolist()
        bMaxPrice_filtered = session['bMaxPrice'][session['bMaxPrice'].notna()].tolist()
        bSumPrice_filtered = session['bSumPrice'][session['bSumPrice'].notna()].tolist()
        bStep_filtered = session['bStep'][session['bStep'].notna()].tolist()
        availability_filtered = session['availability'][session['availability'].notna()].tolist()
        maxVal_filtered = session['maxVal'][session['maxVal'].notna()].tolist()
        customerScore_filtered = session['customerScore'][session['customerScore'].notna()].tolist()
        accountLifetime_filtered = session['accountLifetime'][session['accountLifetime'].notna()].tolist()
        payments_filtered = session['payments'][session['payments'].notna()].tolist()
        age_filtered = session['age'][session['age'].notna()].tolist()
        address_filtered = session['address'][session['address'].notna()].tolist()
        lastOrder_filtered = session['lastOrder'][session['lastOrder'].notna()].tolist()
        
        # Mengisi nilai berdasarkan hasil filter
        tmpSession['cMinPrice'] = min(cMinPrice_filtered) if cMinPrice_filtered else None
        tmpSession['cMaxPrice'] = max(cMaxPrice_filtered) if cMaxPrice_filtered else None
        tmpSession['cSumPrice'] = cSumPrice_filtered[-1] if cSumPrice_filtered else None
        tmpSession['bMinPrice'] = min(bMinPrice_filtered) if bMinPrice_filtered else None
        tmpSession['bMaxPrice'] = max(bMaxPrice_filtered) if bMaxPrice_filtered else None
        tmpSession['bSumPrice'] = bSumPrice_filtered[-1] if bSumPrice_filtered else None
        tmpSession['bStep'] = stats.mode(bStep_filtered)[0][0] if bStep_filtered else None
        tmpSession['availability'] = stats.mode(availability_filtered)[0][0] if availability_filtered else None
        tmpSession['maxVal'] = max(maxVal_filtered) if maxVal_filtered else None
        tmpSession['customerScore'] = sum(customerScore_filtered)/len(customerScore_filtered) if customerScore_filtered else None
        tmpSession['accountLifetime'] = max(accountLifetime_filtered) if accountLifetime_filtered else None
        tmpSession['payments'] = max(payments_filtered) if payments_filtered else None
        tmpSession['age'] = max(age_filtered) if age_filtered else None
        tmpSession['address'] = stats.mode(address_filtered)[0][0] if address_filtered else None
        tmpSession['lastOrder'] = max(lastOrder_filtered) if lastOrder_filtered else None
        
        processed_sessions.append(tmpSession)
    
    allSession = pd.concat(processed_sessions, axis=1).T if processed_sessions else pd.DataFrame()
    return allSession

Fungsi ini dilakukan untuk memproses kumpulan data yang telah dipisahkan menjadi tiap sesi (dalam variabel splitted). Fungsi ini berfokus pada pengisian nilai yang hilang (missing values) dalam setiap sesi berdasarkan beberapa aturan logika dan kemudian menggabungkan hasil pemrosesan untuk setiap sesi menjadi satu dataset.

Pertama kita memulai dengan iterasi untuk setiap sesi dalam daftar splitted. Untuk setiap sesi, fungsi mengambil salinan dari baris terakhir sebagai referensi untuk pengisian nilai-nilai yang hilang.

Kemudian kita filter Data Non-NA dengan cara setiap kolom yang relevan dalam sesi (seperti harga, langkah, status, dan lainnya) akan difilter untuk menghilangkan nilai yang hilang (missing values). Nilai yang tersisa akan disimpan dalam daftar terpisah.

Setelah memfilter nilai yang valid, kita mengisi nilai yang hilang (missing values) pada kolom-kolom tertentu: Untuk kolom harga (misalnya harga minimum dan maksimum), fungsi akan mengisi dengan nilai terkecil atau terbesar dari data yang ada. Untuk kolom dengan data numerik (misalnya skor pelanggan), nilai rata-rata dihitung dan digunakan untuk menggantikan nilai yang hilang. Untuk kolom kategorikal (seperti langkah atau status ketersediaan), fungsi menggunakan nilai yang paling sering muncul (mode) untuk mengganti nilai yang hilang. Beberapa kolom lain diisi dengan nilai terakhir yang tersedia jika ada.

Setiap hasil pemrosesan sesi yang telah diperbarui dengan nilai yang hilang digabungkan kembali ke dalam list processed_sessions. Setelah pemrosesan semua sesi, hasil akhir adalah penggabungan seluruh sesi yang telah diperbarui ke dalam satu DataFrame menggunakan pd.concat.

In [20]:
bySession = groupBy(splitted)

In [21]:
bySession = bySession.drop(columns='customerNo')

In [22]:
bySession['sessionNo'] = bySession['sessionNo'].astype(int)
bySession['startHour'] = bySession['startHour'].astype(int)
bySession['startWeekday'] = bySession['startWeekday'].astype(int)
bySession['duration'] = bySession['duration'].astype(float)
bySession['cCount'] = bySession['cCount'].astype(int)
bySession['cMinPrice'] = pd.to_numeric(bySession['cMinPrice'], errors='coerce')
bySession['cMaxPrice'] = pd.to_numeric(bySession['cMaxPrice'], errors='coerce')
bySession['cSumPrice'] = pd.to_numeric(bySession['cSumPrice'], errors='coerce')
bySession['bCount'] = bySession['bCount'].astype(int)
bySession['bMinPrice'] = pd.to_numeric(bySession['bMinPrice'], errors='coerce')
bySession['bMaxPrice'] = pd.to_numeric(bySession['bMaxPrice'], errors='coerce')
bySession['bSumPrice'] = pd.to_numeric(bySession['bSumPrice'], errors='coerce')
bySession['bStep'] = pd.to_numeric(bySession['bStep'], errors='coerce').fillna(np.nan).astype('Int64')

bySession['onlineStatus'] = bySession['onlineStatus'].map({'y': 1, 'n': 0})
bySession['onlineStatus'] = bySession['onlineStatus'].fillna(-1).astype(int)

bySession['availability'] = bySession['availability'].map({
    'completely not determinable': 1, 
    'completely not orderable': 2,
    'completely orderable': 3,
    'mainly not determinable': 4,
    'mainly not orderable': 5,
    'mainly orderable': 6,
    'mixed': 7
})
bySession['availability'] = bySession['availability'].fillna(-1).astype(int)

bySession['maxVal'] = pd.to_numeric(bySession['maxVal'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['customerScore'] = pd.to_numeric(bySession['customerScore'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['accountLifetime'] = pd.to_numeric(bySession['accountLifetime'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['payments'] = pd.to_numeric(bySession['payments'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['age'] = pd.to_numeric(bySession['age'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['address'] = pd.to_numeric(bySession['address'], errors='coerce').fillna(np.nan).astype('Int64')
bySession['lastOrder'] = pd.to_numeric(bySession['lastOrder'], errors='coerce').fillna(np.nan).astype('Int64')

bySession['order'] = bySession['order'].map({'y': 1, 'n': 0})


In [23]:

# Membuat objek IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Melakukan imputasi pada dataset
data_imputed = imputer.fit_transform(bySession)

# # Mengubah kembali ke DataFrame
bySession = pd.DataFrame(data_imputed, columns=bySession.columns)


In [24]:
bySession.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sessionNo        50000 non-null  float64
 1   startHour        50000 non-null  float64
 2   startWeekday     50000 non-null  float64
 3   duration         50000 non-null  float64
 4   cCount           50000 non-null  float64
 5   cMinPrice        50000 non-null  float64
 6   cMaxPrice        50000 non-null  float64
 7   cSumPrice        50000 non-null  float64
 8   bCount           50000 non-null  float64
 9   bMinPrice        50000 non-null  float64
 10  bMaxPrice        50000 non-null  float64
 11  bSumPrice        50000 non-null  float64
 12  bStep            50000 non-null  float64
 13  onlineStatus     50000 non-null  float64
 14  availability     50000 non-null  float64
 15  maxVal           50000 non-null  float64
 16  customerScore    50000 non-null  float64
 17  accountLifet

In [25]:
bySession['sessionNo'] = bySession['sessionNo'].astype(int)
bySession['startHour'] = bySession['startHour'].astype(int)
bySession['startWeekday'] = bySession['startWeekday'].astype(int)
bySession['cCount'] = bySession['cCount'].astype(int)
bySession['bCount'] = bySession['bCount'].astype(int)
bySession['bStep'] = bySession['bStep'].astype(int)
bySession['onlineStatus'] = bySession['onlineStatus'].astype(int)
bySession['availability'] = bySession['availability'].astype(int)
bySession['customerScore'] = bySession['customerScore'].astype(int)
bySession['accountLifetime'] = bySession['accountLifetime'].astype(int)
bySession['payments'] = bySession['payments'].astype(int)
bySession['age'] = bySession['age'].astype(int)
bySession['address'] = bySession['address'].astype(int)
bySession['lastOrder'] = bySession['lastOrder'].astype(int)
bySession['order'] = bySession['order'].astype(bool)



In [26]:
def correlation_fields(data):
    # diffCounts
    diffCounts = (data['cCount'] - data['bCount']).values

    return diffCounts

In [27]:
diffCounts = correlation_fields(bySession)

In [28]:
def insert_column (data,fields,fields_name):
    index = len(data.columns)-1
    for i in range(len(fields)):
        data.insert(index + i,fields_name[i],fields[i],True)
    return data

In [29]:
fields = [countLog,diffCounts]
fields_name = ['countLog','diffCounts']
bySession = insert_column(bySession,fields,fields_name)

In [30]:
bySession.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sessionNo        50000 non-null  int32  
 1   startHour        50000 non-null  int32  
 2   startWeekday     50000 non-null  int32  
 3   duration         50000 non-null  float64
 4   cCount           50000 non-null  int32  
 5   cMinPrice        50000 non-null  float64
 6   cMaxPrice        50000 non-null  float64
 7   cSumPrice        50000 non-null  float64
 8   bCount           50000 non-null  int32  
 9   bMinPrice        50000 non-null  float64
 10  bMaxPrice        50000 non-null  float64
 11  bSumPrice        50000 non-null  float64
 12  bStep            50000 non-null  int32  
 13  onlineStatus     50000 non-null  int32  
 14  availability     50000 non-null  int32  
 15  maxVal           50000 non-null  float64
 16  customerScore    50000 non-null  int32  
 17  accountLifet

In [48]:
# Misalkan n adalah 22, kita ingin memilih kolom dari kolom pertama hingga kolom ke-22
n = 24
X = bySession.iloc[:, :n]  # Memilih kolom dari kolom pertama hingga kolom ke-n
y = bySession['order']

# Bagi dataset secara stratified untuk menjaga distribusi kelas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, stratify=y)

# Inisialisasi model XGBoost
model = xgb.XGBClassifier()

# Latih model
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Akurasi: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Akurasi: 0.9222
Precision: 0.9244130189699972
Recall: 0.9222
F1-Score: 0.9223023446380004
